# 自製智能中文選字系統  (1)

In [52]:
import sys
sys.version

'3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

## 資料前處理

確認版本為 python3

In [53]:
import re

In [54]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    segments = re.findall(r'[\u4E00-\u9FFF]+',line)
    # END YOUR CODE
    return segments

In [55]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [56]:
segments = []
with open('./wiki_zh_small.txt') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

In [57]:
segments

['英語',
 '英語英語',
 '又稱爲英文',
 '是一種西日耳曼語言',
 '誕生於中世紀早期的英格蘭',
 '如今具有全球通用語的地位',
 '英語',
 '一詞源於遷居英格蘭的日耳曼部落盎格魯',
 '而',
 '盎格魯',
 '得名於臨波羅的海的半島盎格里亞',
 '弗裏西語是與英語最相近的語言',
 '英語詞彙在中世紀早期受到了其他日耳曼族語言的大量影響',
 '後來受羅曼族語言尤其是法語的影響',
 '英語是將近六十個國家唯一的官方語言或官方語言之一',
 '也是全世界最多國家的官方語言',
 '它是英國',
 '愛爾蘭',
 '美國',
 '加拿大',
 '澳大利亞和新西蘭最常用的語言',
 '也在加勒比',
 '非洲及南亞的部分地區被廣泛使用',
 '英語是世界上母語人口第三多的語言',
 '僅次於漢語和西班牙語',
 '英語是學習者最多的第一外語',
 '是聯合國',
 '歐盟和許多其他國際組織的官方語言',
 '它是使用最廣泛的日耳曼族語言',
 '至少',
 '的日耳曼語族使用者說英語',
 '英語有',
 '多年的發展史',
 '公元',
 '世紀',
 '盎格魯',
 '撒克遜人把他們的各種盎格魯',
 '弗裏西語方言帶到了大不列顛島',
 '它們被稱爲古英語',
 '中古英語始於',
 '世紀後期的諾曼征服',
 '這一時期英語受到了法語的影響',
 '世紀初',
 '倫敦對印刷機的採用',
 '欽定版聖經',
 '的出版及元音大推移標誌了近代英語的開端',
 '通過大英帝國對全球的影響',
 '現代英語在',
 '世紀至',
 '世紀中葉傳播到了世界各地',
 '通過各種印刷和電子媒體',
 '隨着美國取得全球超級大國地位',
 '英語已經成爲了國際對話中居領導地位的世界語言',
 '它還是許多地區和行業',
 '如科學',
 '導航',
 '法律等',
 '的通用語',
 '現代英語和很多其他語言相比屈折變化較少',
 '更多地依靠助動詞和語序來表達複雜的時態',
 '體和語氣',
 '以及被動語態',
 '疑問和一些否定',
 '英語的各種口音和方言在發音和音位方面有顯著差異',
 '有時它們的詞彙',
 '語法和拼法也有所不同',
 '但世界各地說英語的人能基本無礙地溝通交流',
 '早期日耳曼人西支部落',
 '盎格魯族',

## Ngram

一開始要先計算字詞出現的次數

In [58]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數
        # YOUR CODE HERE
        for i in range(1, 1 + self.n):
            for segment in segments:
                self.counters[i] += Counter(self._skip(segment, i))

        base_count = sum(dict(self.counters[1]).values())
        self.counters[0] = Counter({'': base_count})
        # END YOUR CODE

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [59]:
counters = Counters(n=3)
counters.fit(segments)

In [60]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371373})

In [62]:
counters[3]

Counter({'英語英': 1,
         '語英語': 1,
         '又稱爲': 9,
         '稱爲英': 2,
         '爲英文': 1,
         '是一種': 79,
         '一種西': 1,
         '種西日': 1,
         '西日耳': 2,
         '日耳曼': 15,
         '耳曼語': 10,
         '曼語言': 2,
         '誕生於': 2,
         '生於中': 4,
         '於中世': 2,
         '中世紀': 20,
         '世紀早': 3,
         '紀早期': 4,
         '早期的': 21,
         '期的英': 1,
         '的英格': 1,
         '英格蘭': 5,
         '如今具': 1,
         '今具有': 1,
         '具有全': 1,
         '有全球': 1,
         '全球通': 1,
         '球通用': 1,
         '通用語': 8,
         '用語的': 1,
         '語的地': 1,
         '的地位': 21,
         '一詞源': 2,
         '詞源於': 2,
         '源於遷': 1,
         '於遷居': 1,
         '遷居英': 1,
         '居英格': 1,
         '格蘭的': 1,
         '蘭的日': 1,
         '的日耳': 4,
         '耳曼部': 1,
         '曼部落': 1,
         '部落盎': 1,
         '落盎格': 1,
         '盎格魯': 13,
         '得名於': 3,
         '名於臨': 1,
         '於臨波': 1,
         '臨波羅': 1,
         '波羅的': 4,
         '羅的海': 3,
     

In [63]:
'說什麼'[-1]

'麼'

In [64]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        # YOUR CODE HERE
        reference = prefix[-(self.n - 1):] if self.n > 1 else ''
        count_referecne = self.minor_counter[reference]
        probs = []
        for key, count in dict(self.major_counter).items():
            if key.startswith(reference):
                prob = count / count_referecne
                probs.append((prob, key[-1]))
        sorted_probs = sorted(probs, reverse=True)
        # END YOUR CODE
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}

In [65]:
unigram = Ngram(1, counters)

In [66]:
# unigram 看來輸入什麼字都是一樣的結果

unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]

[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [67]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [68]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        # YOUR CODE HERE
        if len(prefix) > 1:
            return self.trigram.predict_proba(prefix, top_k)
        elif len(prefix) > 0:
            return self.bigram.predict_proba(prefix, top_k)
        else:
            return self.unigram.predict_proba(prefix, top_k)
        # END YOUR CODE

In [69]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [70]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

## Demo

In [71]:
# !pip install -U pip
# !pip install -q ipywidgets

In [72]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')